CELL 0 — Imports



In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
import os
import datetime
from tqdm.auto import tqdm

/Users/saurabhkumar/miniforge3/envs/spark_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# CELL 1 — Spark session (reuse if already running)

from pyspark.sql import SparkSession

try:
    spark  # noqa: F821
    spark.sparkContext.setLogLevel("ERROR")
    print("Re-using existing Spark session.")
except NameError:
    spark = (
        SparkSession.builder
        .appName("OECD_Synthetic_Parquet_Builder")
        .getOrCreate()
    )
    spark.sparkContext.setLogLevel("ERROR")
    print("Spark session created.")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/14 13:53:58 WARN Utils: Your hostname, Saurabhs-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.5.199 instead (on interface en0)
26/01/14 13:53:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/14 13:53:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/14 13:53:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Spark session created.


In [3]:
# # CELL 2 — Regenerate Synthetic Data with Balanced SOCs and Data-Rich Text

# from pyspark.sql import functions as F
# from pyspark.sql.types import *
# from pyspark.sql.window import Window

# # 1. Spark Configuration
# spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# # -----------------------
# # 1) INPUT / OUTPUT PATHS
# # -----------------------
# INPUT_CSV_PATH = "/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/reed_jobs_uk_extended.csv"
# OUTPUT_PARQUET_PATH = "/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD"

# # ------------------------------------------
# # 2) TARGET SCHEMA
# # ------------------------------------------
# target_schema = StructType([
#     StructField("region_lad_value", StringType(), True),
#     StructField("soc2020_unit_group_title", StringType(), True),
#     # ... (rest of schema as before)
#     StructField("soc_2020", StringType(), True),
#     StructField("full_text", StringType(), True),
#     StructField("date", DateType(), True),
#     StructField("score", FloatType(), True),
#     StructField("imputed_expiration_date", DateType(), True),
#     StructField("job_title", StringType(), True),
#     StructField("unique_job_id", StringType(), True),
#     StructField("job_id", StringType(), True),
#     # Including common fields to avoid selection errors
#     StructField("profession_group", StringType(), True),
#     StructField("MCA", StringType(), True),
#     StructField("location_name", StringType(), True),
#     StructField("guid_ons", StringType(), True),
#     StructField("LEP21NM2", StringType(), True),
#     StructField("profession", StringType(), True),
#     StructField("region_lad_label", StringType(), True),
#     StructField("profession_class", StringType(), True),
#     StructField("region", StringType(), True),
#     StructField("LSIP", StringType(), True),
#     StructField("LEP21NM1", StringType(), True),
#     StructField("run_id", StringType(), True),
#     StructField("region_label", StringType(), True),
#     StructField("posting_id", StringType(), True),
# ])

# target_cols = [f.name for f in target_schema.fields]

# # ------------------------------------------
# # 3) READ CSV
# # ------------------------------------------
# raw = (
#     spark.read
#     .option("header", True)
#     .option("multiLine", True)
#     .option("escape", "\"")
#     .csv(INPUT_CSV_PATH)
# )

# # ------------------------------------------
# # 4) ADD MISSING COLUMNS
# # ------------------------------------------
# for c in target_cols:
#     if c not in raw.columns:
#         raw = raw.withColumn(c, F.lit(None).cast("string"))

# # ------------------------------------------
# # 5) GENERATE EQUAL RANDOM DATES (2020-2025)
# # ------------------------------------------
# start_date = "2020-01-01"
# total_days = 2191
# raw = raw.withColumn("date", F.date_add(F.to_date(F.lit(start_date)), (F.rand(seed=42) * total_days).cast("int")))
# raw = raw.withColumn("imputed_expiration_date", F.date_add(F.col("date"), 30))

# # ------------------------------------------
# # 6) SYNTHETIC BALANCED SOC & DATA-RICH TEXT
# # ------------------------------------------
# # Defining our pool (Mix of Tech and Non-Tech)
# SOC4_POOL = ["2133","2134","2135","2136","2421","2424", # Tech/Data heavy
#              "4112","4121","5245","5243","1137","1254"] # Admin/Manual
# soc_array = F.array(*[F.lit(x) for x in SOC4_POOL])
# pool_size = len(SOC4_POOL)

# # Assign SOCs EQUALLY using Row Number
# w = Window.orderBy(F.rand(seed=42))
# raw = raw.withColumn("row_idx", F.row_number().over(w) - 1)
# raw = raw.withColumn("soc_2020", F.element_at(soc_array, (F.col("row_idx") % pool_size + 1).cast("int")))

# # INJECT DATA TERMS for specific SOCs to ensure the NLP pipeline finds them
# # This appends "Data-Intensive" keywords to the full_text for our target SOCs
# data_keywords = " Keywords: data analytics, python programming, sql database management, machine learning, statistical modeling, dashboard visualization."

# raw = raw.withColumn(
#     "full_text",
#     F.when(
#         F.col("soc_2020").isin("2133","2134","2135","2136","2421","2424"),
#         F.concat(F.coalesce(F.col("full_text"), F.lit("")), F.lit(data_keywords))
#     ).otherwise(F.col("full_text"))
# )

# # ------------------------------------------
# # 7) FINAL SELECTION & SCHEMA ENFORCEMENT
# # ------------------------------------------
# df_out = raw.select(*target_cols)
# for field in target_schema.fields:
#     df_out = df_out.withColumn(field.name, F.col(field.name).cast(field.dataType))

# # ------------------------------------------
# # 8) WRITE TO PARQUET
# # ------------------------------------------
# (
#     df_out
#     .repartition(1) 
#     .write.mode("overwrite")
#     .parquet(OUTPUT_PARQUET_PATH)
# )

# print(f"✅ Success! Generated balanced synthetic data at: {OUTPUT_PARQUET_PATH}")
# print("Counts per SOC (Should be equal):")
# df_out.groupBy("soc_2020").count().show()

CELL 1 — Spark session (reuse if already running)

In [4]:
try:
    spark  # noqa: F821
    spark.sparkContext.setLogLevel("ERROR")
    print("Re-using existing Spark session.")
except NameError:
    auto_config = (
        SparkSession.builder
        .appName("OECD_Data_Intensity_Pipeline")
        .config("spark.executor.memory", "13g")
        .config("spark.driver.memory", "3g")
        .config("spark.executor.cores", "4")
        .config("spark.driver.cores", "4")
        .config("spark.sql.adaptive.enabled", "true")
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
        .config("spark.sql.adaptive.skewJoin.enabled", "true")
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")
        .config("spark.sql.parquet.compression.codec", "snappy")
    )
    spark = auto_config.getOrCreate()
    spark.sparkContext.setLogLevel("ERROR")
    print("Spark session created.")

Re-using existing Spark session.


CELL 2 — Global parameters (NO loose vars later)

In [7]:
# ---------------------------
# Pipeline parameters
# ---------------------------
# "ALL" will auto-detect all years, or specify range for production
YEARS = "2020-2025"   

# Sampling: Set to 1.0 (100%) for your full production run
SAMPLE_FRACTION = 1.0      
SAMPLE_SEED = 42
STRATIFIED_BY_SOC = False   

# NLP thresholds (OECD-style)
# These remain standard
SIM_THRESHOLD = 0.01       
DATA_THRESHOLD = 1         

# How much to store per job
# Keep this at 10. Increasing it (e.g., to 100) will bloat storage on 240M rows.
TOP_K = 10                   

# Re-run behaviour
# Set to False if you want to be able to restart a failed run without re-doing finished years
FORCE_RECOMPUTE = False      

# Debugging
WRITE_DEBUG_CHUNKS = False   
DEBUG_SAMPLE_FRACTION = 0.0001

print("Production parameters set.")

Production parameters set.


CELL 3 — Paths (S3A root + per-year output folders)


In [8]:
# Root directory: keep everything inside OECD_DATA
BASE_S3A_PATH = "/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD"  # e.g. "s3a://onscdp-prd-data01-d4946922/dapsen/workspace_zone/online_job_ads/OECD_DATA"

# If you ever want to read from a single parquet instead of CSV partitions
PARQUET_PATH = "/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/part-00000-6f2787d8-9f9c-4b9b-9903-fc9d83e3d0c0-c000.snappy.parquet"  # e.g. f"{BASE_S3A_PATH}/somefile.parquet"
CSV_PATH = None      # if None, uses BASE_S3A_PATH/csv_data/<year>

def get_available_years():
    return list(range(2020, 2026))

def parse_year_input(year_input):
    available = get_available_years()
    if isinstance(year_input, str):
        y = year_input.strip().upper()
        if y == "ALL":
            years = available
        elif "-" in y:
            s, e = map(int, y.split("-"))
            years = list(range(s, e + 1))
        else:
            years = [int(y)]
    elif isinstance(year_input, (list, tuple)):
        years = [int(x) for x in year_input]
    else:
        years = [int(year_input)]
    bad = [yy for yy in years if yy not in available]
    if bad:
        raise ValueError(f"Invalid years {bad}. Available: {available}")
    return sorted(years)

years_to_process = parse_year_input(YEARS)
print("Years to process:", years_to_process)

def validate_oecd_path(path: str) -> bool:
    if not path.startswith(BASE_S3A_PATH):
        raise ValueError(f"Path {path} is outside OECD_DATA root")
    return True

def get_paths_for_year(year: int):
    base_year_path = os.path.join(BASE_S3A_PATH, "processed_data", str(year))
    input_csv = os.path.join(BASE_S3A_PATH, "csv_data", str(year)) if CSV_PATH is None else os.path.join(CSV_PATH, str(year))

    return {
        # input
        "input_csv": input_csv,

        # outputs (job-level, efficient)
        "job_features": os.path.join(base_year_path, "job_features"),
        "debug_chunks": os.path.join(base_year_path, "debug_chunks"),

        # classification outputs
        "job_categories": os.path.join(base_year_path, "job_categories"),
        "occupation_summary": os.path.join(base_year_path, "occupation_summary"),

        # synthetic industry outputs
        "job_categories_sic": os.path.join(base_year_path, "job_categories_sic"),
        "sector_summary_sic": os.path.join(base_year_path, "sector_summary_sic"),
    }

ALL_PATHS = {yr: get_paths_for_year(yr) for yr in years_to_process}

# Validate
for yr, paths in ALL_PATHS.items():
    for k, p in paths.items():
        validate_oecd_path(p)

print("Path validation complete.")
print("Example year paths:", years_to_process[0], ALL_PATHS[years_to_process[0]])

Years to process: [2020, 2021, 2022, 2023, 2024, 2025]
Path validation complete.
Example year paths: 2020 {'input_csv': '/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/csv_data/2020', 'job_features': '/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2020/job_features', 'debug_chunks': '/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2020/debug_chunks', 'job_categories': '/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2020/job_categories', 'occupation_summary': '/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2020/occupation_summary', 'job_categories_sic': '/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2020/job_categories_sic', 'sector_summary_sic': '/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2020/sector_summary_sic'}


CELL 4 — Safe writing utilities (S3A-safe, versioned, restartable)

In [9]:
def safe_write_parquet_s3a(df, output_base_path, mode="overwrite", create_version=True, verify_read=True):
    """
    Safe-ish write for object stores (s3a://).
    Writes to version folder output_base_path/_v=YYYYMMDD_HHMMSS and updates a _LATEST_POINTER file.
    """
    ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    version_path = f"{output_base_path}/_v={ts}" if create_version else output_base_path

    print(f"[WRITE] parquet -> {version_path}")
    df.write.mode(mode).option("compression", "snappy").parquet(version_path)

    if verify_read:
        _ = spark.read.parquet(version_path).limit(1).count()

    # marker (optional)
    try:
        marker_df = spark.createDataFrame([(ts,)], ["written_at"])
        marker_df.coalesce(1).write.mode("overwrite").json(f"{version_path}/_SUCCESS_MARKER")
    except Exception as e:
        print(f"Warning: marker write failed: {e}")

    # pointer
    if create_version:
        try:
            latest_df = spark.createDataFrame([(version_path, ts)], ["latest_path", "timestamp"])
            latest_df.coalesce(1).write.mode("overwrite").json(f"{output_base_path}/_LATEST_POINTER")
        except Exception as e:
            print(f"Warning: pointer update failed: {e}")

    return version_path

def read_latest_version(output_base_path):
    pointer_path = f"{output_base_path}/_LATEST_POINTER"
    pointer = spark.read.json(pointer_path)
    latest = pointer.orderBy(F.col("timestamp").desc()).limit(1).collect()[0]["latest_path"]
    return spark.read.parquet(latest), latest

print("Utilities ready: safe_write_parquet_s3a(), read_latest_version()")

Utilities ready: safe_write_parquet_s3a(), read_latest_version()


CELL 5 — Step 1: Load data per year (with sampling parameter)
This is where you control 1% / 2% / 10% without touching downstream logic.

In [10]:
all_data = {}
total_jobs = 0

if PARQUET_PATH:
    print(f"[LOAD] unified parquet from {PARQUET_PATH}")
    full_df = spark.read.parquet(PARQUET_PATH).withColumn("date", F.to_date("date"))

    for yr in tqdm(years_to_process, desc="Reading data (Parquet)"):
        df = full_df.filter(F.year("date") == yr)

        if SAMPLE_FRACTION is not None and float(SAMPLE_FRACTION) < 1.0:
            frac = float(SAMPLE_FRACTION)
            df = df.sample(False, frac, seed=SAMPLE_SEED)
            print(f"[LOAD] Year {yr}: sampled {frac*100:.2f}%")

        cnt = df.count()
        all_data[yr] = df
        total_jobs += cnt
        print(f"[LOAD] Year {yr}: {cnt:,} rows")

else:
    for yr in tqdm(years_to_process, desc="Reading data (CSV)"):
        print(f"\n[LOAD] Year {yr} from {ALL_PATHS[yr]['input_csv']} ...")

        df = (
            spark.read
            .option("header", True)
            .option("multiline", True)
            .csv(ALL_PATHS[yr]["input_csv"])
            .select(
                "date",
                "job_id",
                "soc_2020",
                "job_title",
                F.col("full_text").cast("string")
            )
            .filter(F.col("full_text").isNotNull() & (F.length("full_text") > 0))
            .withColumn("date", F.to_date("date", "yyyy-MM-dd"))
        )

        # Sampling
        if SAMPLE_FRACTION is not None and float(SAMPLE_FRACTION) < 1.0:
            frac = float(SAMPLE_FRACTION)

            if STRATIFIED_BY_SOC:
                # approximate stratified sample
                soc_vals = [r["soc_2020"] for r in df.select("soc_2020").distinct().limit(50000).collect()]
                fractions = {s: frac for s in soc_vals if s is not None}
                df = df.sampleBy("soc_2020", fractions=fractions, seed=SAMPLE_SEED)
                print(f"[LOAD] Year {yr}: stratified sample {frac*100:.2f}% by SOC")
            else:
                df = df.sample(False, frac, seed=SAMPLE_SEED)
                print(f"[LOAD] Year {yr}: uniform sample {frac*100:.2f}%")

        cnt = df.count()
        all_data[yr] = df
        total_jobs += cnt
        print(f"[LOAD] Year {yr}: {cnt:,} rows")

print(f"\n[LOAD] Total jobs loaded: {total_jobs:,}")

[LOAD] unified parquet from /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/part-00000-6f2787d8-9f9c-4b9b-9903-fc9d83e3d0c0-c000.snappy.parquet


Reading data (Parquet):  33%|███▎      | 2/6 [00:01<00:02,  1.80it/s]

[LOAD] Year 2020: 1,755 rows
[LOAD] Year 2021: 1,633 rows


Reading data (Parquet):  67%|██████▋   | 4/6 [00:01<00:00,  3.90it/s]

[LOAD] Year 2022: 1,691 rows
[LOAD] Year 2023: 1,710 rows


Reading data (Parquet): 100%|██████████| 6/6 [00:01<00:00,  3.48it/s]

[LOAD] Year 2024: 1,600 rows
[LOAD] Year 2025: 1,611 rows

[LOAD] Total jobs loaded: 10,000


CELL 6 — Step 2: NLP job-level feature extraction (efficient storage)


OECD alignment (adapted)
	•	OECD extracts noun chunks from “pure data” occupations then spreads list across all jobs.
	•	Your efficient adaptation: still extracts noun chunks + similarity to “data”, but stores job-level summaries and top chunks. This keeps the pipeline scalable while preserving interpretability.

In [11]:
import spacy
import pandas as pd
import gc
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, ArrayType

# =========================================================
# 1. SINGLETON MODEL LOADER (Crucial for Speed/RAM)
# =========================================================
# This global variable ensures we load the heavy NLP model only ONCE per Executor,
# preventing memory overflows on 240M rows.
_SPACY_MODEL = None

def get_spacy_model():
    global _SPACY_MODEL
    if _SPACY_MODEL is None:
        print("Loading SpaCy model on executor...")
        try:
            # We exclude components we don't need (ner, lemmatizer) to save RAM
            _SPACY_MODEL = spacy.load("en_core_web_lg", exclude=["lemmatizer", "ner"])
        except OSError:
            # Fallback for lightweight testing if 'lg' is missing
            print("Warning: 'en_core_web_lg' not found, falling back to 'sm'")
            _SPACY_MODEL = spacy.load("en_core_web_sm", exclude=["lemmatizer", "ner"])
    return _SPACY_MODEL

# =========================================================
# 2. SCHEMA DEFINITION
# =========================================================
job_features_schema = StructType([
    StructField("date", StringType()),
    StructField("job_id", StringType()),
    StructField("soc_2020", StringType()),
    StructField("n_chunks_total", IntegerType()),
    StructField("n_chunks_data", IntegerType()),
    StructField("avg_sim_data", DoubleType()),
    # Keeping top_chunks as an Array is much more efficient than exploding to rows
    StructField("top_chunks", ArrayType(StringType())), 
    StructField("top_sims", ArrayType(DoubleType())),
])

# =========================================================
# 3. WORKER FUNCTION
# =========================================================
def extract_job_features_production(iterator):
    # Initialize model once per partition iterator
    nlp = get_spacy_model()
    target = nlp("data") # Pre-compute the target vector

    for pdf in iterator:
        # 1. Basic Cleaning
        pdf["full_text"] = pdf["full_text"].fillna("").astype(str)
        
        # 2. Batch Processing (Much faster than individual loops)
        texts = pdf["full_text"].tolist()
        # n_process=1 is recommended inside Spark executors to avoid deadlock
        docs = nlp.pipe(texts, batch_size=50, n_process=1) 
        
        out_rows = []
        
        # 3. Iterate results
        for doc, job_id, soc, date in zip(docs, pdf["job_id"], pdf["soc_2020"], pdf["date"]):
            try:
                data_chunks = []
                total_chunks = 0
                
                for chunk in doc.noun_chunks:
                    total_chunks += 1
                    # Only check vector if it exists
                    if chunk.has_vector:
                        sim = float(chunk.similarity(target))
                        if sim >= SIM_THRESHOLD:
                            # Truncate strictly to avoid storage bloat on 240M rows
                            cleaned = chunk.text.strip().lower()[:100] 
                            data_chunks.append((cleaned, sim))
                
                # Sort and select Top K
                n_data = len(data_chunks)
                if n_data > 0:
                    data_chunks.sort(key=lambda x: x[1], reverse=True)
                    top_k_items = data_chunks[:TOP_K]
                    top_chunks = [t[0] for t in top_k_items]
                    top_sims = [t[1] for t in top_k_items]
                    avg_sim = sum(s for _, s in data_chunks) / n_data
                else:
                    top_chunks, top_sims, avg_sim = [], [], 0.0

                out_rows.append({
                    "date": str(date),
                    "job_id": str(job_id),
                    "soc_2020": str(soc),
                    "n_chunks_total": total_chunks,
                    "n_chunks_data": n_data,
                    "avg_sim_data": avg_sim,
                    "top_chunks": top_chunks,
                    "top_sims": top_sims
                })
            except Exception as e:
                # Log error but do not crash the huge job for one bad row
                continue
            
        yield pd.DataFrame(out_rows)

# =========================================================
# 4. EXECUTION LOOP (With Memory Management)
# =========================================================
for yr in tqdm(years_to_process, desc="NLP job feature extraction"):
    out_base = ALL_PATHS[yr]["job_features"]
    
    # Check if exists (Skip logic to allow restarting)
    if not FORCE_RECOMPUTE:
        try:
            read_latest_version(out_base)
            print(f"\n[SKIP] Year {yr} job_features already exist.")
            continue
        except Exception:
            pass

    print(f"\n[NLP] Year {yr} starting...")
    
    # Load data for specific year
    df = all_data[yr]

    # --- PARTITIONING STRATEGY ---
    # For 12 million rows/year (240M total / 20 years), 
    # 600 partitions = ~20,000 rows per task. 
    # This is the "Sweet Spot" for heavy NLP tasks.
    df_part = df.repartition(600) 

    # Run UDF
    job_feat = df_part.mapInPandas(extract_job_features_production, schema=job_features_schema)
    
    # Add partition columns for efficient writing
    job_feat = job_feat.withColumn("year", F.year(F.to_date("date"))) \
                       .withColumn("month", F.month(F.to_date("date")))

    # Write WITHOUT reading back (verify_read=False saves huge time)
    latest_written = safe_write_parquet_s3a(
        job_feat,
        out_base,
        mode="overwrite",
        create_version=True,
        verify_read=False 
    )

    print(f"[NLP] Year {yr} written to: {latest_written}")
    
    # --- MEMORY CLEANUP (Critical for 20-year loops) ---
    job_feat.unpersist()
    del df_part
    del job_feat
    gc.collect() # Force driver to release RAM

NLP job feature extraction:  33%|███▎      | 2/6 [00:00<00:00,  5.48it/s]


[SKIP] Year 2020 job_features already exist.

[SKIP] Year 2021 job_features already exist.


NLP job feature extraction:  67%|██████▋   | 4/6 [00:00<00:00,  7.00it/s]


[SKIP] Year 2022 job_features already exist.

[SKIP] Year 2023 job_features already exist.


NLP job feature extraction: 100%|██████████| 6/6 [00:00<00:00,  6.92it/s]


[SKIP] Year 2024 job_features already exist.

[SKIP] Year 2025 job_features already exist.


CELL 7 — Step 3: Job classification + Occupation summary (SOC4)
Digit level answer: This produces occupation estimates at SOC 4-digit (soc4 = first 4 chars of soc_2020).

In [12]:
# Landmark SOC4 groups
data_entry_soc = {"4111","4112","4113","4114","4121","4131","4132","4150"}
database_soc = {"2423","2136"}
data_analytics_soc = {"2421","2424","2133","2135"}

for yr in tqdm(years_to_process, desc="Job classification + occ summary"):
    print(f"\n[CLASSIFY] Year {yr}")

    # Read the features we just created
    job_feat_df, job_feat_path = read_latest_version(ALL_PATHS[yr]["job_features"])
    
    # Ensure we only process this year's data
    job_feat_df = job_feat_df.filter(F.year(F.to_date("date")) == yr)
    
    # Extract SOC4 (First 4 characters)
    job_feat_df = job_feat_df.withColumn("soc4", F.substring(F.col("soc_2020"), 1, 4))

    # Apply Threshold Rule
    is_data_intensive = (F.col("n_chunks_data") >= F.lit(DATA_THRESHOLD))

    # Create Categories
    job_categories = (
        job_feat_df.select("job_id","soc_2020","soc4","date","n_chunks_data","avg_sim_data","top_chunks","top_sims")
        .withColumn("data_entry", (F.col("soc4").isin(list(data_entry_soc)) & is_data_intensive).cast("int"))
        .withColumn("database", (F.col("soc4").isin(list(database_soc)) & is_data_intensive).cast("int"))
        .withColumn("data_analytics", (F.col("soc4").isin(list(data_analytics_soc)) & is_data_intensive).cast("int"))
        .withColumn("any_data_intensive", is_data_intensive.cast("int"))
        .withColumn("year", F.year(F.to_date("date")))
        .withColumn("month", F.month(F.to_date("date")))
    )

    # Save job-level categories (Lazy check for existence)
    jc_base = ALL_PATHS[yr]["job_categories"]
    if not FORCE_RECOMPUTE:
        try:
            read_latest_version(jc_base)
            print(f"[SKIP] job_categories exists for {yr}")
            # We must load it if we skipped computation to support the next step? 
            # Actually, for the summary below, it's safer to just re-compute the aggregation 
            # from the features since aggregation is fast.
        except:
             safe_write_parquet_s3a(job_categories, jc_base, create_version=True, verify_read=False)
    else:
        safe_write_parquet_s3a(job_categories, jc_base, create_version=True, verify_read=False)

    # Aggregation: Occupation summary (SOC4)
    # Note: We rely on the dataframe 'job_categories' which is lazy. 
    # Spark will optimize the read from job_features directly.
    occ = (
        job_categories.groupBy("soc4")
        .agg(
            F.count("*").alias("total_jobs"),
            F.sum("data_entry").alias("data_entry_jobs"),
            F.sum("database").alias("database_jobs"),
            F.sum("data_analytics").alias("data_analytics_jobs"),
            F.sum("any_data_intensive").alias("any_data_intensive_jobs"),
        )
        .withColumn("data_entry_share", 100 * F.col("data_entry_jobs") / F.col("total_jobs"))
        .withColumn("database_share", 100 * F.col("database_jobs") / F.col("total_jobs"))
        .withColumn("data_analytics_share", 100 * F.col("data_analytics_jobs") / F.col("total_jobs"))
        .withColumn("total_data_share", 100 * F.col("any_data_intensive_jobs") / F.col("total_jobs"))
        .withColumn("year", F.lit(int(yr)))
    )

    os_base = ALL_PATHS[yr]["occupation_summary"]
    safe_write_parquet_s3a(occ, os_base, create_version=True, verify_read=False)

    print(f"[CLASSIFY] Year {yr} completed.")

Job classification + occ summary:   0%|          | 0/6 [00:00<?, ?it/s]


[CLASSIFY] Year 2020
[SKIP] job_categories exists for 2020
[WRITE] parquet -> /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2020/occupation_summary/_v=20260114_135527


Job classification + occ summary:  17%|█▋        | 1/6 [00:02<00:13,  2.71s/it] 

[CLASSIFY] Year 2020 completed.

[CLASSIFY] Year 2021
[SKIP] job_categories exists for 2021
[WRITE] parquet -> /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2021/occupation_summary/_v=20260114_135529


Job classification + occ summary:  33%|███▎      | 2/6 [00:04<00:08,  2.10s/it]

...

[CLASSIFY] Year 2023 completed.

[CLASSIFY] Year 2024
[SKIP] job_categories exists for 2024
[WRITE] parquet -> /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2024/occupation_summary/_v=20260114_135534


Job classification + occ summary:  83%|████████▎ | 5/6 [00:09<00:01,  1.77s/it]

[CLASSIFY] Year 2024 completed.

[CLASSIFY] Year 2025
[SKIP] job_categories exists for 2025
[WRITE] parquet -> /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2025/occupation_summary/_v=20260114_135536


Job classification + occ summary: 100%|██████████| 6/6 [00:11<00:00,  1.86s/it]

[CLASSIFY] Year 2025 completed.


CELL 8 — Step 4: Synthetic SIC Section (Option 1 rule-based SOC→SIC)
This produces industry at SIC “Section” level (A–U style) — same level OECD often reports when rolling up to broad industries.

In [13]:
# Synthetic SIC mapping (DEMO ONLY)
# Rule: SOC major group (first digit) -> SIC Section letter
SOC_MAJOR_TO_SIC = {
    "1": "M",  # managers -> professional/technical proxy
    "2": "M",
    "3": "J",  # associate prof -> info/comm proxy
    "4": "N",  # admin -> admin/support proxy
    "5": "F",  # skilled trades -> construction proxy
    "6": "Q",  # caring -> health/social proxy
    "7": "G",  # sales -> wholesale/retail proxy
    "8": "C",  # process/plant -> manufacturing proxy
    "9": "N",  # elementary -> admin/support proxy
}

def soc_to_sic_section(soc):
    if soc is None:
        return None
    s = str(soc).strip()
    if len(s) == 0:
        return None
    return SOC_MAJOR_TO_SIC.get(s[0], None)

sic_udf = F.udf(soc_to_sic_section, StringType())

for yr in tqdm(years_to_process, desc="Synthetic SIC + sector summary"):
    print(f"\n[SIC-SYNTH] Year {yr}")

    jc_df, jc_path = read_latest_version(ALL_PATHS[yr]["job_categories"])
    jc_df = jc_df.filter(F.year(F.to_date("date")) == yr)
    print(f"[SIC-SYNTH] Using job_categories from: {jc_path}")

    jc_sic = jc_df.withColumn("SICSection_synth", sic_udf(F.col("soc_2020")))

    # Save job-level with SIC
    jc_sic_base = ALL_PATHS[yr]["job_categories_sic"]
    if FORCE_RECOMPUTE:
        jc_sic_written = safe_write_parquet_s3a(jc_sic, jc_sic_base, create_version=True)
    else:
        try:
            _df, latest = read_latest_version(jc_sic_base)
            print(f"[SKIP] job_categories_sic exists: {latest}")
            jc_sic_written = latest
        except Exception:
            jc_sic_written = safe_write_parquet_s3a(jc_sic, jc_sic_base, create_version=True)

    # Sector summary by synthetic SIC section
    sec = (
        jc_sic.filter(F.col("SICSection_synth").isNotNull())
        .groupBy("SICSection_synth")
        .agg(
            F.count("*").alias("total_jobs"),
            F.sum("data_entry").alias("data_entry_jobs"),
            F.sum("database").alias("database_jobs"),
            F.sum("data_analytics").alias("data_analytics_jobs"),
            F.sum("any_data_intensive").alias("any_data_intensive_jobs"),
        )
        .withColumn("data_entry_share", 100 * F.col("data_entry_jobs") / F.col("total_jobs"))
        .withColumn("database_share", 100 * F.col("database_jobs") / F.col("total_jobs"))
        .withColumn("data_analytics_share", 100 * F.col("data_analytics_jobs") / F.col("total_jobs"))
        .withColumn("total_data_share", 100 * F.col("any_data_intensive_jobs") / F.col("total_jobs"))
        .withColumn("year", F.lit(int(yr)))
    )

    sec_base = ALL_PATHS[yr]["sector_summary_sic"]
    if FORCE_RECOMPUTE:
        sec_written = safe_write_parquet_s3a(sec, sec_base, create_version=True)
    else:
        try:
            _df, latest = read_latest_version(sec_base)
            print(f"[SKIP] sector_summary_sic exists: {latest}")
            sec_written = latest
        except Exception:
            sec_written = safe_write_parquet_s3a(sec, sec_base, create_version=True)

    print(f"[SIC-SYNTH] Saved year {yr}: job_categories_sic={jc_sic_written} | sector_summary={sec_written}")

Synthetic SIC + sector summary:   0%|          | 0/6 [00:00<?, ?it/s]


[SIC-SYNTH] Year 2020
[SIC-SYNTH] Using job_categories from: /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2020/job_categories/_v=20260114_115744


Synthetic SIC + sector summary:  17%|█▋        | 1/6 [00:00<00:02,  2.39it/s]

[SKIP] job_categories_sic exists: /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2020/job_categories_sic/_v=20260114_115812
[SKIP] sector_summary_sic exists: /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2020/sector_summary_sic/_v=20260114_115814
[SIC-SYNTH] Saved year 2020: job_categories_sic=/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2020/job_categories_sic/_v=20260114_115812 | sector_summary=/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2020/sector_summary_sic/_v=20260114_115814

[SIC-SYNTH] Year 2021
[SIC-SYNTH] Using job_categories from: /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2021/job_categories/_v=20260114_115749
[SKIP] job_categories_sic exists: /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2021/job_categories_sic/_v=20260114_115817


Synthetic SIC + sector summary:  33%|███▎      | 2/6 [00:00<00:01,  2.59it/s]

...

[SKIP] job_categories_sic exists: /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2023/job_categories_sic/_v=20260114_115827
[SKIP] sector_summary_sic exists: /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2023/sector_summary_sic/_v=20260114_115829
[SIC-SYNTH] Saved year 2023: job_categories_sic=/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2023/job_categories_sic/_v=20260114_115827 | sector_summary=/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2023/sector_summary_sic/_v=20260114_115829

[SIC-SYNTH] Year 2024
[SIC-SYNTH] Using job_categories from: /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2024/job_categories/_v=20260114_115803
[SKIP] job_categories_sic exists: /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2024/job_categories_sic/_v=20260114_115832


Synthetic SIC + sector summary:  83%|████████▎ | 5/6 [00:01<00:00,  2.81it/s]

[SKIP] sector_summary_sic exists: /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2024/sector_summary_sic/_v=20260114_115834
[SIC-SYNTH] Saved year 2024: job_categories_sic=/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2024/job_categories_sic/_v=20260114_115832 | sector_summary=/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2024/sector_summary_sic/_v=20260114_115834

[SIC-SYNTH] Year 2025
[SIC-SYNTH] Using job_categories from: /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2025/job_categories/_v=20260114_115807


Synthetic SIC + sector summary: 100%|██████████| 6/6 [00:02<00:00,  2.72it/s]

[SKIP] job_categories_sic exists: /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2025/job_categories_sic/_v=20260114_115837
[SKIP] sector_summary_sic exists: /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2025/sector_summary_sic/_v=20260114_115839
[SIC-SYNTH] Saved year 2025: job_categories_sic=/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2025/job_categories_sic/_v=20260114_115837 | sector_summary=/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2025/sector_summary_sic/_v=20260114_115839


CELL 9 — Step 5: Load outputs into pandas for visualisations

In [14]:
import pandas as pd
import plotly.express as px

occ_frames = []
sec_frames = []

for yr in years_to_process:
    occ_df, occ_path = read_latest_version(ALL_PATHS[yr]["occupation_summary"])
    sec_df, sec_path = read_latest_version(ALL_PATHS[yr]["sector_summary_sic"])

    occ_frames.append(occ_df.toPandas())
    sec_frames.append(sec_df.toPandas())

occupation_df = pd.concat(occ_frames, ignore_index=True)
sector_df = pd.concat(sec_frames, ignore_index=True)

occupation_df["year"] = occupation_df["year"].astype(int)
sector_df["year"] = sector_df["year"].astype(int)

print("Loaded occupation_df:", occupation_df.shape)
print("Loaded sector_df:", sector_df.shape)

Loaded occupation_df: (72, 11)
Loaded sector_df: (18, 11)


CELL 10 — Step 6: Core plots (multi-year)


A) overall trend

In [15]:
yearly = (
    occupation_df.groupby("year")[["total_jobs","any_data_intensive_jobs"]]
    .sum()
    .reset_index()
)
yearly["data_intensive_share"] = 100 * yearly["any_data_intensive_jobs"] / yearly["total_jobs"]

fig = px.line(
    yearly.sort_values("year"),
    x="year",
    y="data_intensive_share",
    markers=True,
    title="Share of data-intensive job adverts over time",
    labels={"year":"Year", "data_intensive_share":"Data-intensive jobs (%)"}
)
fig.show()

B) SOC4 top occupations per year (facets)

In [16]:
top_n = 20
top_each_year = (
    occupation_df.sort_values(["year","total_data_share"], ascending=[True,False])
    .groupby("year")
    .head(top_n)
)

melted = top_each_year.melt(
    id_vars=["year","soc4"],
    value_vars=["data_entry_share","database_share","data_analytics_share"],
    var_name="category",
    value_name="share"
)

label_map = {
    "data_entry_share":"Data entry",
    "database_share":"Database",
    "data_analytics_share":"Data analytics"
}
melted["category_label"] = melted["category"].map(label_map)

fig = px.bar(
    melted,
    x="soc4",
    y="share",
    color="category_label",
    facet_col="year",
    facet_col_wrap=2,
    title=f"Top {top_n} occupations by data intensity — each year (stacked components)",
    labels={"soc4":"SOC (4-digit)", "share":"Share (%)", "category_label":"Layer"},
    height=900
)
fig.update_layout(barmode="stack")
fig.show()

C) Synthetic SIC sector view (multi-year)

In [17]:
fig = px.line(
    sector_df.sort_values(["year","SICSection_synth"]),
    x="year",
    y="total_data_share",
    color="SICSection_synth",
    markers=True,
    title="Synthetic SIC Section — data-intensive share over time",
    labels={"year":"Year","total_data_share":"Data-intensive share (%)","SICSection_synth":"Synthetic SIC Section"}
)
fig.show()

In [18]:
# NEW CELL: Find the most 'data-like' jobs regardless of threshold
from pyspark.sql import functions as F

# Load the job features we just wrote
df_features, _ = read_latest_version(ALL_PATHS[2024]["job_features"])

print("Top 10 jobs most related to 'data' by average similarity:")
df_features.filter(F.col("avg_sim_data").isNotNull()) \
           .orderBy(F.col("avg_sim_data").desc()) \
           .select("job_id", "avg_sim_data", "n_chunks_data", "top_chunks") \
           .show(10, truncate=False)

Top 10 jobs most related to 'data' by average similarity:
+------+------------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------+
|job_id|avg_sim_data      |n_chunks_data|top_chunks                                                                                                                              |
+------+------------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------+
|None  |0.4927734306880406|7            |[data analytics, sql database management, statistical modeling, dashboard visualization, machine learning, python programming, keywords]|
|None  |0.4927734306880406|7            |[data analytics, sql database management, statistical modeling, dashboard visualization, machine learning, python programming, keywords]|
|None  |0.4927734306880406|7            |[data 

CELL 11 — STEP 7 — OECD-STYLE VALUATION USING SUT (BENCHMARK YEAR = 2023)

In [19]:
# ============================================================
# STEP 7 — OECD Valuation: Data Investment from Job Ad Shares
# ============================================================
# Logic:
# 1) Use sector-level data-intensity shares from job ads (2020–2025)
# 2) Map to grouped SIC sections
# 3) Value labour using 2023 SUT (COMP_EMP, GVA)
# 4) Apply OECD alpha capitalisation parameters
# 5) Produce data investment as % of GVA
#
# OECD reference:
# "What is the role of data in jobs..." (OECD, 2023)
# ============================================================

from pyspark.sql import functions as F
from pyspark.sql.types import *

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------
SUT_YEAR = 2023
SUT_CSV_PATH = "/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/SUT_TABLE.csv"  # <-- update

ALPHA_DATA_ENTRY = 0.25
ALPHA_DATABASE = 0.50
ALPHA_DATA_ANALYTICS = 0.75

# ------------------------------------------------------------
# 1) READ SUT (2023 benchmark)
# ------------------------------------------------------------
sut = (
    spark.read
    .option("header", True)
    .csv(SUT_CSV_PATH)
    .withColumn("year", F.col("year").cast("int"))
    .withColumn("SIC_Code", F.col("SIC_Code").cast("string"))
    .withColumn("GVA_basic_prices", F.col("GVA_basic_prices").cast("double"))
    .withColumn("COMP_EMP", F.col("COMP_EMP").cast("double"))
    .filter(F.col("year") == F.lit(SUT_YEAR))
)

# ------------------------------------------------------------
# 2) LOAD SECTOR SUMMARY (ALL YEARS)
# ------------------------------------------------------------
sec_frames = []
for yr in years_to_process:
    df, _ = read_latest_version(ALL_PATHS[yr]["sector_summary_sic"])
    sec_frames.append(df)

sector_all = sec_frames[0]
for df in sec_frames[1:]:
    sector_all = sector_all.unionByName(df, allowMissingColumns=True)

sector_all = sector_all.withColumn("year", F.col("year").cast("int"))

# ------------------------------------------------------------
# 3) NORMALISE SIC LETTERS → SUT GROUPS
# ------------------------------------------------------------
def grouped_sic(col):
    c = F.upper(F.trim(col))
    return (
        F.when(c.isin("B","C","D","E"), F.lit("B-E"))
         .when(c.isin("G","H","I"), F.lit("G-I"))
         .when(c.isin("M","N"), F.lit("M-N"))
         .when(c.isin("O","P","Q"), F.lit("O-Q"))
         .when(c.isin("R","S","T"), F.lit("R-T"))
         .otherwise(c)
    )

sector_all = sector_all.withColumn(
    "SIC_Code",
    grouped_sic(F.col("SICSection_synth"))
)

# ------------------------------------------------------------
# 4) WEIGHTED RE-AGGREGATION (CRITICAL FIX)
# ------------------------------------------------------------
sector_grp = (
    sector_all
    .groupBy("year", "SIC_Code")
    .agg(
        (F.sum(F.col("data_entry_share") * F.col("total_jobs")) /
         F.sum("total_jobs")).alias("data_entry_share"),

        (F.sum(F.col("database_share") * F.col("total_jobs")) /
         F.sum("total_jobs")).alias("database_share"),

        (F.sum(F.col("data_analytics_share") * F.col("total_jobs")) /
         F.sum("total_jobs")).alias("data_analytics_share"),

        (F.sum(F.col("total_data_share") * F.col("total_jobs")) /
         F.sum("total_jobs")).alias("total_data_share"),

        F.sum("total_jobs").alias("total_jobs")
    )
)

# ------------------------------------------------------------
# 5) JOIN TO SUT (2023 BENCHMARK, APPLIED TO ALL YEARS)
# ------------------------------------------------------------
valued = (
    sector_grp
    .join(
        sut.select("SIC_Code", "GVA_basic_prices", "COMP_EMP"),
        on="SIC_Code",
        how="inner"
    )
)

# ------------------------------------------------------------
# 6) OECD LABOUR VALUATION
# ------------------------------------------------------------
valued = (
    valued
    .withColumn("p_data_entry", F.col("data_entry_share") / 100)
    .withColumn("p_database", F.col("database_share") / 100)
    .withColumn("p_data_analytics", F.col("data_analytics_share") / 100)
)

valued = (
    valued
    .withColumn("data_entry_labour_value",
                F.col("p_data_entry") * F.col("COMP_EMP"))
    .withColumn("database_labour_value",
                F.col("p_database") * F.col("COMP_EMP"))
    .withColumn("data_analytics_labour_value",
                F.col("p_data_analytics") * F.col("COMP_EMP"))
)

# ------------------------------------------------------------
# 7) APPLY OECD α CAPITALISATION
# ------------------------------------------------------------
valued = (
    valued
    .withColumn("data_entry_investment",
                F.lit(ALPHA_DATA_ENTRY) * F.col("data_entry_labour_value"))
    .withColumn("database_investment",
                F.lit(ALPHA_DATABASE) * F.col("database_labour_value"))
    .withColumn("data_analytics_investment",
                F.lit(ALPHA_DATA_ANALYTICS) * F.col("data_analytics_labour_value"))
)

valued = valued.withColumn(
    "total_data_investment",
    F.col("data_entry_investment")
    + F.col("database_investment")
    + F.col("data_analytics_investment")
)

# ------------------------------------------------------------
# 8) OECD HEADLINE METRIC
# ------------------------------------------------------------
valued = valued.withColumn(
    "data_investment_share_of_GVA",
    100 * F.col("total_data_investment") / F.col("GVA_basic_prices")
)

# ------------------------------------------------------------
# 9) FINAL OUTPUT
# ------------------------------------------------------------
final_cols = [
    "year", "SIC_Code",
    "GVA_basic_prices", "COMP_EMP",
    "data_entry_share", "database_share", "data_analytics_share",
    "total_data_share",
    "total_data_investment",
    "data_investment_share_of_GVA"
]

final_valuation = valued.select(*final_cols).orderBy("year", "SIC_Code")

print("STEP 7 COMPLETE — OECD-style valuation output:")
final_valuation.show(truncate=False)

STEP 7 COMPLETE — OECD-style valuation output:


+----+--------+----------------+--------+----------------+------------------+--------------------+-----------------+---------------------+----------------------------+
|year|SIC_Code|GVA_basic_prices|COMP_EMP|data_entry_share|database_share    |data_analytics_share|total_data_share |total_data_investment|data_investment_share_of_GVA|
+----+--------+----------------+--------+----------------+------------------+--------------------+-----------------+---------------------+----------------------------+
|2020|F       |148457.0        |60509.0 |0.0             |0.0               |0.0                 |0.0              |0.0                  |0.0                         |
|2020|M-N     |340424.0        |215043.0|0.0             |9.979494190020505 |40.054682159945315  |60.08202323991798|75331.1944634313     |22.12863795250373           |
|2021|F       |148457.0        |60509.0 |0.0             |0.0               |0.0                 |0.0              |0.0                  |0.0                   

# STEP 8 — TIME SERIES OF DATA INVESTMENT (2020–2025)



This step analyses how the estimated value of data-related investment in labour evolves over time.
Using job-advert-derived data-intensity shares for each year (2020–2025) and valuing them against the 2023 Supply-Use Table, we construct a time series of data investment as a share of Gross Value Added (GVA).
This isolates structural change in data use within jobs, holding industry prices and structures constant.

In [20]:
# ============================================================
# STEP 8 — Time Series: Data Investment as % of GVA (2020–2025)
# ============================================================

import pyspark.sql.functions as F
import pandas as pd
import plotly.express as px

# ------------------------------------------------------------
# 1) USE OUTPUT FROM STEP 7
# ------------------------------------------------------------
# Assumes `final_valuation` exists from Step 7

ts_df = final_valuation.select(
    "year",
    "SIC_Code",
    "total_data_investment",
    "GVA_basic_prices",
    "data_investment_share_of_GVA"
)

# ------------------------------------------------------------
# 2) AGGREGATE TO ECONOMY-WIDE SERIES
# ------------------------------------------------------------
economy_ts = (
    ts_df
    .groupBy("year")
    .agg(
        F.sum("total_data_investment").alias("total_data_investment"),
        F.sum("GVA_basic_prices").alias("total_GVA")
    )
    .withColumn(
        "data_investment_share_of_GVA",
        100 * F.col("total_data_investment") / F.col("total_GVA")
    )
    .orderBy("year")
)

print("Economy-wide data investment time series:")
economy_ts.show(truncate=False)

# ------------------------------------------------------------
# 3) CONVERT TO PANDAS FOR VISUALISATION
# ------------------------------------------------------------
economy_pd = economy_ts.toPandas()
economy_pd["year"] = economy_pd["year"].astype(int)

# ------------------------------------------------------------
# 4) LINE CHART — ECONOMY-WIDE
# ------------------------------------------------------------
fig = px.line(
    economy_pd,
    x="year",
    y="data_investment_share_of_GVA",
    markers=True,
    title="Data investment as a share of GVA (economy-wide)",
    labels={
        "year": "Year",
        "data_investment_share_of_GVA": "Data investment (% of GVA)"
    }
)

fig.update_layout(
    yaxis_tickformat=".1f",
    template="plotly_white"
)

fig.show()

# ------------------------------------------------------------
# 5) SECTOR-LEVEL TIME SERIES (OPTIONAL BUT POWERFUL)
# ------------------------------------------------------------
sector_pd = ts_df.toPandas()
sector_pd["year"] = sector_pd["year"].astype(int)

fig2 = px.line(
    sector_pd,
    x="year",
    y="data_investment_share_of_GVA",
    color="SIC_Code",
    markers=True,
    title="Data investment as a share of GVA by sector (synthetic SIC)",
    labels={
        "year": "Year",
        "data_investment_share_of_GVA": "Data investment (% of GVA)",
        "SIC_Code": "SIC Section"
    }
)

fig2.update_layout(
    yaxis_tickformat=".1f",
    template="plotly_white"
)

fig2.show()

Economy-wide data investment time series:
+----+---------------------+---------+----------------------------+
|year|total_data_investment|total_GVA|data_investment_share_of_GVA|
+----+---------------------+---------+----------------------------+
|2020|75331.1944634313     |488881.0 |15.408902056621406          |
|2021|75288.75055106539    |488881.0 |15.400220207180354          |
|2022|75242.15684882896    |488881.0 |15.39068952338687           |
|2023|75302.77684210528    |488881.0 |15.403089267552897          |
|2024|75216.65322580645    |488881.0 |15.385472789044053          |
|2025|75297.0742740134     |488881.0 |15.401922814348154          |
+----+---------------------+---------+----------------------------+



# STEP 9 — α SENSITIVITY BANDS (OECD-STYLE)

This table reports economy-wide estimates of data investment as a share of Gross Value Added (GVA) under alternative capitalisation assumptions (α).
The Low, Central, and High scenarios reflect conservative, baseline, and optimistic assumptions regarding the proportion of data-related labour that is capitalised as investment, following OECD practice.



In [21]:
# ============================================================
# STEP 9 — Alpha Sensitivity Analysis (TABLE + CHART)
# Requires: `valued` from Step 7 (industry-level with labour values + GVA)
# ============================================================

from pyspark.sql import functions as F
import pandas as pd
import plotly.express as px

# ------------------------------------------------------------
# 1) Define alpha scenarios (OECD-style bands)
# ------------------------------------------------------------
ALPHA_SCENARIOS = {
    "Low": {
        "data_entry": 0.15,
        "database": 0.35,
        "data_analytics": 0.55,
    },
    "Central": {
        "data_entry": 0.25,
        "database": 0.50,
        "data_analytics": 0.75,
    },
    "High": {
        "data_entry": 0.35,
        "database": 0.65,
        "data_analytics": 0.95,
    },
}

# ------------------------------------------------------------
# 2) Build sensitivity TABLE (economy-wide per year)
# ------------------------------------------------------------
sens_tables = []

for scenario, alpha in ALPHA_SCENARIOS.items():

    df = (
        valued
        .withColumn(
            "total_data_investment",
            F.lit(alpha["data_entry"]) * F.col("data_entry_labour_value")
            + F.lit(alpha["database"]) * F.col("database_labour_value")
            + F.lit(alpha["data_analytics"]) * F.col("data_analytics_labour_value")
        )
        .groupBy("year")
        .agg(
            F.sum("total_data_investment").alias("total_data_investment"),
            F.sum("GVA_basic_prices").alias("total_GVA")
        )
        .withColumn(
            "data_investment_share_of_GVA",
            100.0 * F.col("total_data_investment") / F.col("total_GVA")
        )
        .withColumn("alpha_scenario", F.lit(scenario))
    )

    sens_tables.append(df)

sens_all = sens_tables[0]
for df in sens_tables[1:]:
    sens_all = sens_all.unionByName(df)

sens_all = sens_all.orderBy("year", "alpha_scenario")

print("STEP 9 — Alpha sensitivity table (Spark):")
sens_all.show(truncate=False)

# ------------------------------------------------------------
# 3) Convert to Pandas for plotting + easy sharing
# ------------------------------------------------------------
sens_pd = sens_all.toPandas()
sens_pd["year"] = sens_pd["year"].astype(int)

# Optional: ensure sensible ordering
order_map = {"Low": 0, "Central": 1, "High": 2}
sens_pd["scenario_order"] = sens_pd["alpha_scenario"].map(order_map)
sens_pd = sens_pd.sort_values(["year", "scenario_order"]).drop(columns=["scenario_order"])

print("\nSTEP 9 — Alpha sensitivity table (Pandas):")
display(sens_pd)

# ------------------------------------------------------------
# 4) CHART: time series with three lines
# ------------------------------------------------------------
fig_lines = px.line(
    sens_pd,
    x="year",
    y="data_investment_share_of_GVA",
    color="alpha_scenario",
    markers=True,
    title="Data investment as % of GVA (α sensitivity band, economy-wide)",
    labels={
        "year": "Year",
        "data_investment_share_of_GVA": "Data investment (% of GVA)",
        "alpha_scenario": "α scenario",
    }
)
fig_lines.show()

# ------------------------------------------------------------
# 5) CHART: ribbon band (Low–High) with Central line
# ------------------------------------------------------------
wide = (
    sens_pd
    .pivot(index="year", columns="alpha_scenario", values="data_investment_share_of_GVA")
    .reset_index()
)

# Safety: only proceed if all three exist
needed = {"Low", "Central", "High"}
if needed.issubset(set(wide.columns)):
    fig_band = px.line(
        wide,
        x="year",
        y=["Low", "Central", "High"],
        markers=True,
        title="Data investment as % of GVA (Low–High band + Central)",
        labels={"value": "Data investment (% of GVA)", "variable": "α scenario", "year": "Year"}
    )
    fig_band.show()
else:
    print("Warning: Missing one of Low/Central/High columns after pivot, cannot draw band chart.")

STEP 9 — Alpha sensitivity table (Spark):
+----+---------------------+---------+----------------------------+--------------+
|year|total_data_investment|total_GVA|data_investment_share_of_GVA|alpha_scenario|
+----+---------------------+---------+----------------------------+--------------+
|2020|75331.1944634313     |488881.0 |15.408902056621406          |Central       |
|2020|95777.18304853041    |488881.0 |19.591103570916115          |High          |
|2020|54885.20587833219    |488881.0 |11.2267005423267            |Low           |
|2021|75288.75055106539    |488881.0 |15.400220207180354          |Central       |
|2021|95726.5257531227     |488881.0 |19.58074168419773           |High          |
|2021|54850.975349008084   |488881.0 |11.219698730162982          |Low           |
|2022|75242.15684882896    |488881.0 |15.39068952338687           |Central       |
|2022|95662.84769339958    |488881.0 |19.567716416346634          |High          |
|2022|54821.46600425835    |488881.0 |11.2136

,year,total_data_investment,total_GVA,data_investment_share_of_GVA,alpha_scenario
2,2020,54885.205878,488881.0,11.226701,Low
0,2020,75331.194463,488881.0,15.408902,Central
1,2020,95777.183049,488881.0,19.591104,High
5,2021,54850.975349,488881.0,11.219699,Low
3,2021,75288.750551,488881.0,15.400220,Central
4,2021,95726.525753,488881.0,19.580742,High
8,2022,54821.466004,488881.0,11.213663,Low
6,2022,75242.156849,488881.0,15.390690,Central
7,2022,95662.847693,488881.0,19.567716,High
11,2023,54862.373789,488881.0,11.222030,Low
